# 🧠 Support Vector Machines (SVM) - Complete Guide with Titanic Dataset

---

## 🔷 What is SVM?

Support Vector Machine (SVM) is a **supervised learning algorithm** used for **classification** and **regression**.

SVM finds the **optimal hyperplane** that separates classes with the **maximum margin**.

---

## ✅ Intuition

SVM aims to:

- Separate classes using a **hyperplane**
- **Maximize the margin** between the nearest data points of each class
- Use only the **support vectors** (the closest points) to define the decision boundary

A larger margin usually means better generalization to new data.

---

## 🧮 Mathematics of Linear SVM

For binary classification with labels $y_i \in \{-1, 1\}$ and data points $x_i \in \mathbb{R}^n$, the primal form of the optimization problem is:

$
\min_{w, b} \ \frac{1}{2} \|w\|^2
$

subject to:

$
y_i (w \cdot x_i + b) \geq 1 \quad \text{for all } i
$

- $w$ is the normal vector to the hyperplane
- $b$ is the bias term
- The margin is $\frac{2}{\|w\|}$

If the data is not linearly separable, we introduce **slack variables** $\xi_i$ and a **regularization parameter** $C$:

$
\min_{w, b, \xi} \ \frac{1}{2} \|w\|^2 + C \sum_{i=1}^n \xi_i
$

---

## 🌐 Nonlinear Data: The Kernel Trick

To deal with nonlinear boundaries, SVM uses **kernels** to project data into a higher-dimensional space.

The kernel function $K(x_i, x_j) = \phi(x_i)^T \phi(x_j)$ computes inner products in that space **without explicitly transforming the data**.

Common kernels:

- **Linear**: $K(x, x') = x^T x'$
- **Polynomial**: $K(x, x') = (x^T x' + 1)^d$
- **RBF (Gaussian)**: $K(x, x') = \exp(-\gamma \|x - x'\|^2)$

---

## 🔑 Key Parameters in SVM

| Parameter | Description |
|----------|-------------|
| `C` | Regularization parameter: trade-off between margin width and misclassification |
| `kernel` | Defines the kernel type (e.g., `'linear'`, `'rbf'`) |
| `gamma` | Controls kernel complexity in RBF and polynomial kernels |

---

# 🧪 Real-World Example: Predicting Titanic Survival with SVM

---

In [ ]:
## 📦 1. Load and Explore Data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# Load Titanic dataset
df = sns.load_dataset('titanic')

In [ ]:
df = df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]
df.dropna(inplace=True)

In [ ]:
## 🧹 2. Preprocess the Data
# Encode categorical features
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df['embarked'] = df['embarked'].map({'S': 0, 'C': 1, 'Q': 2})

# Features and target
X = df.drop('survived', axis=1)
y = df['survived']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
## 🤖 3. Train and Evaluate SVM Model
model = SVC(kernel='rbf', C=10, gamma='scale')
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# Evaluation
print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_estimator(model, X_test_scaled, y_test)
plt.show()

In [ ]:
## 🛠️ 4. Hyperparameter Tuning with Grid Search
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.01, 0.1, 1],
    'kernel': ['rbf']
}

grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
print("Best parameters:", grid.best_params_)

In [ ]:
y_pred = grid.predict(X_test_scaled)

# Evaluation
print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_estimator(grid, X_test_scaled, y_test)
plt.show()